# Cypher e Análise em Redes

Entre no Sandbox do Cypher: https://neo4j.com/

Abaixo estão alguns comandos executados no laboratório de interação entre medicamentos:

## Comandos Básicos Cypher

~~~cypher
CREATE ()

MATCH (n)
RETURN n

CREATE ({name:"Acetylsalicylic Acid"})

CREATE ({name:"Dipyrone"})

MATCH (n {name:"Dipyrone"})
RETURN n

MATCH (n)
DELETE n

CREATE (:Drug {name:"Acetylsalicylic Acid"})

CREATE (:Drug {name:"Dipyrone"})

CREATE (:Pathology {name:"Headache"})

MATCH (n)
RETURN n

MATCH (d:Drug)
RETURN d

CREATE (:Drug {name:"Paracetamol"})-[:Treats]->(:Pathology {name:"Fever"})

MATCH (d:Drug {name:"Dipyrone"})
MATCH (p:Pathology {name:"Fever"})
CREATE (d)-[:Treats]->(p)

MATCH (n1)-[e]->(n2)
DELETE e

MATCH (n)
DELETE n
~~~

Comandos avançados Cypher:

~~~cypher
LOAD CSV WITH HEADERS FROM 'https://github.com/santanche/lab2learn/raw/master/data/faers-2017/drug.csv' AS line
CREATE (:Drug { code: line.code, name: line.name})

CREATE INDEX ON :Drug(code)

LOAD CSV WITH HEADERS FROM 'https://github.com/santanche/lab2learn/raw/master/data/faers-2017/pathology.csv' AS line
CREATE (:Pathology { code: line.code, name: line.name})

CREATE INDEX ON :Pathology(code)

LOAD CSV WITH HEADERS FROM 'https://github.com/santanche/lab2learn/raw/master/data/faers-2017/drug-use.csv' AS line
MATCH (d:Drug {code: line.codedrug})
MATCH (p:Pathology {code: line.codepathology})
CREATE (d)-[:Treats {person: line.idperson}]->(p)

MATCH (d)-[:Treats]->(p)
RETURN d, p
LIMIT 50

MATCH (d:Drug)-[t:Treats]->(p:Pathology)
DELETE t

LOAD CSV WITH HEADERS FROM 'https://github.com/santanche/lab2learn/raw/master/data/faers-2017/drug-use.csv' AS line
MATCH (d:Drug {code: line.codedrug})
MATCH (p:Pathology {code: line.codepathology})
MERGE (d)-[t:Treats]->(p)
ON CREATE SET t.weight=1
ON MATCH SET t.weight=t.weight+1

MATCH (d)-[t:Treats]->(p)
WHERE t.weight > 50
RETURN d,p

MATCH (d)-[t:Treats]->(p)
WHERE t.weight > 20
RETURN d,p

MATCH (d1:Drug)-[a]->(p:Pathology)<-[b]-(d2:Drug)
WHERE a.weight > 20 AND b.weight > 20
MERGE (d1)<-[r:Relates]->(d2)
ON CREATE SET r.weight=1
ON MATCH SET r.weight=r.weight+1
~~~

# NHANES Analysis of Fishbone Variables

In this notebook, we will analyze the correlation among variables and profiles of individuals examined in NHANES survey. The focus is on four variables known as Fishbone: WBC, Hgb, HCT, and PLT (see below).  This work is based on the research of Patrícia Raia Nogueira Cavoto.

### Selecting the driver

The sentence below defines the driver for the database.

In [3]:
%defaultDatasource jdbc:h2:mem:db

# Reference values for NHANES for the 2005-2006 survey

* Extracted from data of the NHANES Web site (https://wwwn.cdc.gov/nchs/nhanes/).

## Importing normal ranges of values indicated in the NHANES documentation

The following query imports the file `reference-ranges.csv` that contains reference ranges from NHANES.

* For each variable it is indicated
  - applicable gender
  - age range (ageStart until ageEnd)

* The range is indicated in the form of mininum and maximum values considered normal.

In [4]:
DROP TABLE IF EXISTS ReferenceRanges;
CREATE TABLE ReferenceRanges (
  variable VARCHAR(8),
  gender VARCHAR(1),
  ageStart SMALLINT,
  ageEnd SMALLINT,
  min DECIMAL(7,1),
  max DECIMAL(7,1),
  PRIMARY KEY(variable,gender,ageStart,ageEnd)
) AS SELECT
  variable,gender,ageStart,ageEnd,min,max
FROM CSVREAD('../../data/nhanes2005-2006/reference-ranges.csv');

SELECT DISTINCT variable FROM ReferenceRanges;
SELECT * FROM ReferenceRanges;

org.h2.jdbc.JdbcSQLException:  IO Exception

# Survey NHANES 2005-2006

* Extracted from data of the NHANES Web site (https://wwwn.cdc.gov/nchs/nhanes/).

## Importing data from the survey NHANES 2005-2006

* The following query imports the `combined-selected-variables.csv` that contains a tuple for each individual, with a selected set of variables that are used do diagnose anemia, as mentioned in Figure 1. It was filtered only the individuals with values for all fields.

* We selected four commonly used blood test variables (known as Fishbone).

In [5]:
DROP TABLE IF EXISTS Survey;

CREATE TABLE Survey (
  SEQN VARCHAR(8),
  RIAGENDR VARCHAR(1),
  RIDAGEYR SMALLINT,
  LBXWBCSI DECIMAL(7,1),
  LBXHGB DECIMAL(7,1),
  LBXHCT DECIMAL(7,1),
  LBXPLTSI DECIMAL(7,1),
  PRIMARY KEY(SEQN)
) AS SELECT
  SEQN,RIAGENDR,RIDAGEYR,LBXWBCSI,LBXHGB,LBXHCT,LBXPLTSI
FROM CSVREAD('../../data/nhanes2005-2006/combined-selected-variables.csv');

SELECT COUNT(*) FROM Survey;
SELECT * FROM Survey;

org.h2.jdbc.JdbcSQLException:  IO Exception

# Codes and description of NHANES variables

* The codes and description of variables are stored in a table.

In [6]:
DROP TABLE IF EXISTS VariableDescription;
CREATE TABLE VariableDescription (
  variable VARCHAR(8),
  acronym VARCHAR(8),
  name VARCHAR(50),
  unit VARCHAR(30),
  file VARCHAR(20),
  ranges VARCHAR(100),
  PRIMARY KEY(variable)
) AS SELECT
  variable,acronym,name,unit,file,ranges
FROM CSVREAD('../../data/nhanes2005-2006/reference-ranges-variables.csv');

SELECT * FROM VariableDescription;

org.h2.jdbc.JdbcSQLException:  IO Exception

## Exercício

Escreva sentenças em Cypher que importem a tabela de survey no Neo4J. Como você organizaria esses dados em um grafo para análise? Que métricas você extrairia?

~~~cypher
LOAD CSV WITH HEADERS FROM 'https://github.com/santanche/lab2learn/raw/master/data/nhanes2005-2006/combined-selected-variables.csv' AS line
CREATE (:Survey {code:line.SEQN, gender:toInteger(line.RIAGENDR), age:toInteger(line.RIDAGEYR), whiteCell:toFloat(line.LBXWBCSI), hemoglobin:toFloat(line.LBXHGB), hematocrit:toFloat(line.LBXHCT), platelet:toFloat(line.LBXPLTSI)})
~~~

~~~cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/santanche/lab2learn/master/data/nhanes2005-2006/reference-ranges.csv' AS line
CREATE (:VariableDescription {variable:line.variable,gender:toInteger(line.gender),ageStart:toInteger(line.ageStart),ageEnd:toInteger(line.ageEnd),min:toFloat(line.min),max:toFloat(line.max)})
~~~


Organizaria os dados ligando os nós criado de Survey com os nós de VariableDescription sempre que os valores estiverem normais. Osc
